# PROYECTO MUSIC STREAM

ANÁLISIS DE POPULARIDAD DE CANCIONES EN LA ERA DIGITAL

El objetivo es identificar las canciones y álbumes más populares en MusicStream, utilizando técnicas de extracción de datos desde diferentes fuentes, como la API de Spotify, y last.fm. Posteriormente, almacenaremos la información en una base de datos y realizaremos consultas para obtener insights clave.

LIBRERÍAS Y AUTENTIFICACIONES

In [1]:
# importamos las librerías que necesitaremos:

# Uso de API's
# -----------------------------------------------------------------------
import requests

# Importar librería para interactuar con la API de Spotify y generar los credenciales necesarios para la autentificación 
# -----------------------------------------------------------------------
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Importar librería para trabajar con variables de entorno en Python, permite cargar variables de entorno desde un archivo .env en el directorio del proyecto
# -----------------------------------------------------------------------
import os
from dotenv import load_dotenv

# Importar librería para trabajar con archivos JSON
# -----------------------------------------------------------------------
import json

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
# Importar librerías para procesamiento de texto
# -----------------------------------------------------------------------
import re


# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


In [2]:
load_dotenv()

Python-dotenv could not parse statement starting at line 2


True

In [ ]:
sp_clave = os.getenv("Spotify_id")
sp_contraseña = os.getenv("Spotify_secret")

In [4]:
#Autentificación SPOTIFY:
auth_manager = sp = SpotifyClientCredentials(client_id=sp_clave,
                                             client_secret=sp_contraseña)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
#Autentificación LASTFM:
lastfm_clave = os.getenv("lastfm_id")

FUNCIÓN SACAR CANCIONES DE SPOTIFY

In [6]:
def canciones_spotify(sp, géneros):
    canciones = {"nombre_artista": [], "nombre_canción": [], "año_lanzamiento": [], "género_musical": [], "tipo": []}
    for anio in range(2019,2024):
        for offset in range(0, 250, 50):
            for genero in géneros:
                datos_trance = sp.search(q=f"genre:{genero} year:{anio}", type="track", limit=50, offset=offset)
                for cancion in datos_trance["tracks"]["items"]:
                    canciones["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
                    canciones["nombre_canción"].append(cancion["name"])
                    canciones["género_musical"].append(f"{genero}")
                    canciones["año_lanzamiento"].append(cancion["album"]["release_date"][0:4])
                    canciones["tipo"].append(cancion["type"])
    df_canciones_spotify = pd.DataFrame(canciones)
    return df_canciones_spotify

In [7]:
géneros = ['pop','rock','techno','trance']
df_canciones_spotify = canciones_spotify(sp, géneros)

In [8]:
# guardamos la información de las 5000 canciones de artistas de Spotify en un archivo .csv
df_canciones_spotify.to_csv("spotifydatabase.csv")

In [9]:
df_datos_spotify=pd.read_csv("spotifydatabase.csv",index_col=0)
# ponemos el index_col= 0 para q no nos cree el Unnamed

In [10]:
df_datos_spotify
# visualizamos que es correcta la información de Spotify (el índice va a salir pero lo quitamos al hacer el itertuples)

,nombre_artista,nombre_canción,año_lanzamiento,género_musical,tipo
0,Various Artists,Walk Me Home,2019,pop,track
1,Various Artists,Wow.,2019,pop,track
2,Various Artists,Déjame Ser,2019,pop,track
3,Marisol,"Corre, Corre Caballito - Remasterizado",2019,pop,track
4,Various Artists,I Was Made for Dancin',2019,pop,track
...,...,...,...,...,...
4995,Alexey Sonar,A Tour - Simos Tagias Remix - Mixed,2023,trance,track
4996,Axel Terblanche,Perpetuate,2023,trance,track
4997,Global Cee,"Love, Dream & Energy",2023,trance,track
4998,Various Artists,Wanting You - Extended Mix,2023,trance,track


In [11]:
list(df_datos_spotify.itertuples(index=False, name=None))
# aqui no incluye ni el índice ni los títulos de las colmnas

values_spotify = list(df_datos_spotify.itertuples(index=False, name=None))

FUNCIÓN SACAR CANCIONES DE LASTFM

In [12]:
# leemos el archivo .csv previamente guardado con la información de Spotify
pd.read_csv("spotifydatabase.csv", index_col=0)

,nombre_artista,nombre_canción,año_lanzamiento,género_musical,tipo
0,Various Artists,Walk Me Home,2019,pop,track
1,Various Artists,Wow.,2019,pop,track
2,Various Artists,Déjame Ser,2019,pop,track
3,Marisol,"Corre, Corre Caballito - Remasterizado",2019,pop,track
4,Various Artists,I Was Made for Dancin',2019,pop,track
...,...,...,...,...,...
4995,Alexey Sonar,A Tour - Simos Tagias Remix - Mixed,2023,trance,track
4996,Axel Terblanche,Perpetuate,2023,trance,track
4997,Global Cee,"Love, Dream & Energy",2023,trance,track
4998,Various Artists,Wanting You - Extended Mix,2023,trance,track


In [13]:
leer_spotify_database = pd.read_csv("spotifydatabase.csv")
spotify_database = leer_spotify_database["nombre_artista"].tolist()

In [ ]:
def obtener_info_artistas_lastfm(spotify_database, lastfm_clave):

    infoartistas = {"nombre_artista": [], "bio": [], "listeners": [], "playcount": [], "similar": []}

    for nombreartista in spotify_database:

        if nombreartista == "Various Artists":  # eliminamos Various Artists para evitar datos erróneos
            continue

        if nombreartista in infoartistas["nombre_artista"]:
            continue  # artista ya procesado, lo saltamos

        else: 
            url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={nombreartista}&api_key={lastfm_clave}&format=json"

            respuesta = requests.get(url)

            if respuesta.status_code != 200:
                print(f"❌ Error {respuesta.status_code} para {nombreartista}: {respuesta.reason}")
                continue

            try:
                data = respuesta.json()
                similares = []
                for artistaejemplo in data["artist"]["similar"]["artist"]:
                    similares.append(artistaejemplo['name'])

                infoartistas["similar"].append(similares)
                infoartistas["nombre_artista"].append(nombreartista)
                infoartistas["bio"].append(data["artist"]["bio"]["summary"])
                infoartistas["listeners"].append(data["artist"]["stats"]["listeners"])
                infoartistas["playcount"].append(data["artist"]["stats"]["playcount"])

            except (KeyError, ValueError) as e:
                print(f"⚠️ Datos incompletos o error para {nombreartista}: {e}")
                continue

    return infoartistas

In [ ]:
infoartistas_lastfm = obtener_info_artistas_lastfm(spotify_database, lastfm_clave)
print(infoartistas_lastfm)

# Aunque le hayamos solicitado a LastFM información sobre 5000 canciones de distintos artistas, LastFM no siempre tiene información sobre esos artistas,
# por lo que algunos artistas no van a tener su información sobre listeners, bio o artistas similares (Datos incompletos o error)

⚠️ Datos incompletos o error para Kuber Geetpariha: 'artist'
⚠️ Datos incompletos o error para Kishor Sunar: 'artist'
⚠️ Datos incompletos o error para Pankaj,Mitali Ghosh,Pawan: 'artist'
⚠️ Datos incompletos o error para Mr. Man & the Big Eyes: 'artist'
⚠️ Datos incompletos o error para Pankaj,Monika,Pawan: 'artist'
⚠️ Datos incompletos o error para Dj Rupendra Style: 'artist'
⚠️ Datos incompletos o error para Harendar,Monika,Azad Ansari: 'artist'
⚠️ Datos incompletos o error para Azad Ansari,Manju Devi: 'artist'
⚠️ Datos incompletos o error para Azad Ansari,Manju Devi: 'artist'
⚠️ Datos incompletos o error para KC & The Sunshine Band: 'artist'
⚠️ Datos incompletos o error para Dandi & Ugo: 'artist'
⚠️ Datos incompletos o error para Pawan, Pankaj,Monika: 'artist'
⚠️ Datos incompletos o error para Pankaj,Monika,Pawan: 'artist'
⚠️ Datos incompletos o error para Manoj Sahri,Monika: 'artist'
⚠️ Datos incompletos o error para Pankaj,Pawan: 'artist'
⚠️ Datos incompletos o error para Bharti 

In [20]:
df=pd.DataFrame(infoartistas_lastfm)

In [21]:
df

,nombre_artista,bio,listeners,playcount,similar
0,Marisol,"Marisol (o Pepa Flores, su nombre real) es una...",46644,293612,"[Massiel, Los Secretos, Mocedades, Nino Bravo,..."
1,Da Igual,Wences Sánchez empieza a despuntar desde bien ...,723,8850,"[Barrio Sur, nada que decir, Doctor Pitangú, E..."
2,Los Rebeldes,Los Rebeldes is a Spanish Rock and Roll / Rock...,23226,160468,"[Gatos Locos, Loquillo y Trogloditas, Loquillo..."
3,Duo Dinamico,"<a href=""https://www.last.fm/music/+noredirec...",15196,79109,[]
4,Kañasur,"<a href=""https://www.last.fm/music/Ka%C3%B1as...",173,1642,"[Cherezade, El Suso, El biri, María Artés Lamo..."
...,...,...,...,...,...
1941,Laura Rose,There are several artists with the moniker Lau...,33,372,[]
1942,Matias Chilano,"<a href=""https://www.last.fm/music/Matias+Chi...",2034,12693,"[Breeze & Quadrat, Mariano Favre, MSZ, Loquai,..."
1943,Katty Heath,Katty Heath is an English vocalist. Indie & El...,3190,15223,"[Sue McLaren, Elles De Graaf, Cathy Burton, Be..."
1944,Andy Woldman,"<a href=""https://www.last.fm/music/Andy+Woldm...",1289,3479,"[Glynn Alan, Nucrise, Blake Jarrell, Mercurial..."


In [22]:
# guardamos la información en un archivo .csv sobre los 1946 artistas que aparecen en LastFm:
df.to_csv("lastfmdatabase.csv") 

In [23]:
df_values_lastfm=pd.read_csv("lastfmdatabase.csv",index_col=0)

In [24]:
values_lastfm = list(df_values_lastfm.itertuples(index=False, name=None))
# aqui no incluye ni el índice ni los títulos de las colmnas

FUNCIÓN CREAR BASES DE DATOS:

In [25]:
def crear_base_datos(nombre_bd, user='root', password='AlumnaAdalab', host='127.0.0.1'):

    try:
        cnx = mysql.connector.connect(user=user, password=password, host=host)
        mycursor = cnx.cursor()
        sql = f"CREATE DATABASE {nombre_bd}"
        mycursor.execute(sql)
        print(f"✅ Base de datos `{nombre_bd}` creada correctamente.")
    except mysql.connector.Error as err:
        print(f"❌ Error al crear la base de datos `{nombre_bd}`.")
        print("Error Code:", err.errno)
        print("SQLSTATE:", err.sqlstate)
        print("Message:", err.msg)
    finally:
        mycursor.close()
        cnx.close()

In [26]:
nombre_bd = "MusicTrends"
crear_base_datos(nombre_bd)

✅ Base de datos `MusicTrends` creada correctamente.


FUNCIÓN CREAR TABLAS:

In [27]:
def crear_tablas(queries, user='root', password='AlumnaAdalab', host='127.0.0.1', database='MusicTrends'):

    try:
        cnx = mysql.connector.connect(user=user, password=password, host=host, database=database)
        mycursor = cnx.cursor()

        for sql in queries:
            mycursor.execute(sql)
            cnx.commit()
            print(f"✅ Tabla creada")

    except mysql.connector.Error as err:
        print(f"❌ Error al crear tablas.")
        print("Error Code:", err.errno)
        print("SQLSTATE:", err.sqlstate)
        print("Message:", err.msg)
    finally:
        mycursor.close()
        cnx.close()

In [28]:
queries = [
        """
        CREATE TABLE IF NOT EXISTS LastFM (
            id_artista INT AUTO_INCREMENT NOT NULL,
            nombre_artista VARCHAR(100) NOT NULL,
            bio LONGTEXT NULL,
            listeners INT NULL,
            playcount INT NULL,
            similar LONGTEXT NULL,
            PRIMARY KEY (id_artista)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS Spotify (
            id_track INT NOT NULL AUTO_INCREMENT,
            nombre_canción VARCHAR(200) NULL,
            nombre_artista VARCHAR(100) NOT NULL,
            id_artista INT NULL,
            año_lanzamiento VARCHAR(4),
            género_musical VARCHAR(45) NULL,
            tipo VARCHAR(10) NULL,
            PRIMARY KEY(id_track),
            CONSTRAINT fk_LastFM FOREIGN KEY (id_artista) 
                REFERENCES LastFM(id_artista) 
                ON DELETE CASCADE 
                ON UPDATE CASCADE
        )
        """
    ]
crear_tablas(queries)

✅ Tabla creada
✅ Tabla creada


FUNCIÓN INSERTAR DATOS:

Al crear las tablas hemos definido:
- PRIMARY KEY (id_artista) en la tabla LastFM
- PRIMARY KEY(id_track) en la tabla Spotify y REFERENCES LastFM(id_artista), por lo que Spotify(id_artista) depende de  LastFM(id_artista)


Como los datos de id_artista de Spotify dependen de los id_artista de LastFM, tenemos que realizar las cargas (función insertar_datos) comenzando siempre por la carga de valores de LastFM + la query de LastFM. A continuación obtenemos los artistas que tenemos en ésta tabla de LastFM con la función obtener_diccionario_artistas y los convertimos en valores_spotify con la función preparar_valores_spotify. Una vez tenemos estos valores_spotify podremos realizar la carga de datos en Spotify con la función insertar_datos y la carga de valores de Spotify + la query de Spotify

In [29]:
def insertar_datos(sql, valores, user='root', password='AlumnaAdalab', host='127.0.0.1', database='MusicTrends'):

    try:
        cnx = mysql.connector.connect(user=user, password=password, host=host, database=database)
        mycursor = cnx.cursor()
        mycursor.executemany(sql, valores)
        cnx.commit()
        print(f"✅ {mycursor.rowcount} registro(s) insertado(s).")
    except mysql.connector.Error as err:
        print(f"❌ Error al insertar datos.")
        print("Error Code:", err.errno)
        print("SQLSTATE:", err.sqlstate)
        print("Message:", err.msg)
    finally:
        mycursor.close()
        cnx.close()


In [30]:
# para llamar a la función es necsario darle la query de LastFM y los values de LastFM:
sql = "INSERT INTO LastFM (nombre_artista, bio, listeners, playcount, similar) VALUES (%s, %s, %s, %s,%s)"
valores = values_lastfm
insertar_datos(sql, valores)

✅ 1946 registro(s) insertado(s).


In [31]:
def obtener_diccionario_artistas(user='root', password='AlumnaAdalab', host='127.0.0.1', database='MusicTrends'):
    try:
        cnx = mysql.connector.connect(user=user, password=password, host=host, database=database)
        mycursor = cnx.cursor()
        mycursor.execute("SELECT id_artista, nombre_artista FROM LastFM")
        artistas = mycursor.fetchall()
        diccionario_artistas = {nombre.lower(): id for id, nombre in artistas}
        print(f"✅ Diccionario de {len(diccionario_artistas)} artistas cargado.")
        return diccionario_artistas
    except mysql.connector.Error as err:
        print(f"❌ Error al obtener artistas.")
        print("Error Code:", err.errno)
        print("SQLSTATE:", err.sqlstate)
        print("Message:", err.msg)
        return {}
    finally:
        mycursor.close()
        cnx.close()

In [32]:
diccionario_artistas = obtener_diccionario_artistas()

✅ Diccionario de 1946 artistas cargado.


In [33]:
def preparar_valores_spotify(valores_spotify, diccionario_artistas):
    values_spotify = []
    for fila in valores_spotify:
        nombre_artista = fila[0].lower()
        id_artista = diccionario_artistas.get(nombre_artista)

        if id_artista is not None:
            values_spotify.append((id_artista, *fila))
        else:
            values_spotify.append((None, *fila))
            print(f"⚠️ Artista no encontrado en LastFM: {nombre_artista}")
    return values_spotify

In [34]:
valores_spotify = preparar_valores_spotify(values_spotify, diccionario_artistas)

⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista no encontrado en LastFM: various artists
⚠️ Artista n

In [35]:
# y una vez realizadas las siguientes 2 funciones podremos cargar los datos de Spotify con la función insertar_datos y:
sql = "INSERT INTO Spotify (id_artista, nombre_artista, nombre_canción, año_lanzamiento, género_musical, tipo) VALUES (%s, %s, %s, %s, %s, %s)"
valores = valores_spotify
insertar_datos(sql, valores)

✅ 5000 registro(s) insertado(s).
